In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [ ]:
# Mendefinisikan path dataset
train_data_dir = '/content/drive/MyDrive/dataset/train'
validation_data_dir = '/content/drive/MyDrive/dataset/validation'

In [ ]:
# Mendefinisikan parameter training
batch_size = 32
epochs = 10
img_width, img_height = 224, 224
num_classes = 4

In [ ]:
# Membuat objek pre-trained model ResNet50 tanpa lapisan fully connected (include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
# Membekukan parameter pre-trained model
base_model.trainable = False


In [ ]:
# Membangun model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Mengkompilasi model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Membuat augmentasi gambar
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Mempersiapkan data latih
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 500 images belonging to 4 classes.


In [ ]:
# Mempersiapkan data validasi
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 121 images belonging to 4 classes.


In [ ]:
# Melatih model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

Epoch 1/10
15/15 [==============================] - 162s 10s/step - loss: 1.4635 - accuracy: 0.3440 - val_loss: 1.3603 - val_accuracy: 0.4271
Epoch 2/10
15/15 [==============================] - 167s 11s/step - loss: 1.4216 - accuracy: 0.3632 - val_loss: 1.3561 - val_accuracy: 0.3542
Epoch 3/10
15/15 [==============================] - 177s 12s/step - loss: 1.3351 - accuracy: 0.4209 - val_loss: 1.3298 - val_accuracy: 0.4167
Epoch 4/10
15/15 [==============================] - 166s 11s/step - loss: 1.3556 - accuracy: 0.3547 - val_loss: 1.3548 - val_accuracy: 0.4167
Epoch 5/10
15/15 [==============================] - 169s 11s/step - loss: 1.3786 - accuracy: 0.4042 - val_loss: 1.3316 - val_accuracy: 0.4167
Epoch 6/10
15/15 [==============================] - 171s 12s/step - loss: 1.3621 - accuracy: 0.4038 - val_loss: 1.3732 - val_accuracy: 0.3958
Epoch 7/10
15/15 [==============================] - 166s 11s/step - loss: 1.3346 - accuracy: 0.4167 - val_loss: 1.3222 - val_accuracy: 0.4062
Epoch 

In [ ]:
# Menyimpan model
model.save('/content/drive/MyDrive/model/skin_disease_classification_model.h5')